#**BASIC SETUP**

##**INSTALL DEPENDENCIES & IMPORT LIBS**

In [1]:
#1 Install libs
!pip install langchain openai tiktoken langchain-community pdfplumber sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.

In [2]:
#2 Import libs
import os
import re
import pandas as pd
import json

##**CONNECT DRIVE**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**SETUP API KEY**

In [4]:
from google.colab import userdata
ANTHROPIC_API_KEY = userdata.get("ANTHROPIC_API_KEY")
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
QDRANT_API_KEY = userdata.get("QDRANT_API_KEY")
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
HUGGINGFACE_API_KEY = userdata.get("HUGGINGFACE_API_KEY")
JINAAI_API_KEY = userdata.get("JINAAI_API_KEY")

#**DATA PREP (LOAD PDF, CLEANING, CHUNKING, GENERATE CONTEXT)**

#**DOCUMENTS LOADING AND CHUNKING**

**LOAD DOCUMENT**

In [31]:
#USING PDF_PLUMBER
import pdfplumber
def load_pdf(pdf_path):
    """
    Load a PDF file using pdfplumber and return its text content.

    Args:
        pdf_path (str): Path to the PDF file.

    Returns:
        str: The full text of the PDF with pages joined by newlines.
    """
    text_content = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text() or ""  # Handle None case by using empty string
            text_content.append(text)

    # Combine all pages into a single text, separated by newlines
    full_text = "\n".join(text_content)
    return full_text

**HELPER FUNCTIONS**

In [32]:
def extract_headline(article_text):
    """
    Extract headline from either:
    - 'Điều X. <headline>'
    - 'PHỤ LỤC <số/tên>'
    """

    article_text = article_text.strip()

    # Match "Điều X. <headline>"
    match_dieu = re.match(r"^(Điều\s+\d+\.\s*[^\n\d]+)", article_text)
    if match_dieu:
        return match_dieu.group(1).strip()

    # Match "PHỤ LỤC I", "PHỤ LỤC II - MÔ TẢ...", etc.
    match_appendix = re.match(r"^(PHỤ LỤC\s+[IVXLCDM]+(?:\s*[-–]\s*[^\n]+)?)", article_text, re.IGNORECASE)
    if match_appendix:
        return match_appendix.group(1).strip()

    # Fallback: return the first line
    return article_text.split('\n')[0].strip()


In [33]:
def generate_reference_id(index, doc_code, prefix="D"):
    return f"{doc_code}_{prefix}{index}"

**SPLIT DOCUMENT BY SECTIONS (CHƯƠNG)**

In [34]:
def split_document_into_sections(document_text: str):
    """
    Split the document into Chương sections only (ignore PHỤ LỤC).
    Returns a list of dicts with 'section_id', 'title', 'content', and 'combine'.
    """
    # Match lines like "Chương I\nTÊN CHƯƠNG"
    chuong_pattern = r"(Chương\s+[IVXLCDM]+[\.:]?\s*\n[^\n]*)"

    matches = list(re.finditer(chuong_pattern, document_text, flags=re.IGNORECASE))

    sections = []
    for idx, match in enumerate(matches):
        start = match.start()
        end = matches[idx + 1].start() if idx + 1 < len(matches) else len(document_text)
        block = document_text[start:end].strip()

        # Extract title and content
        lines = block.split("\n", 2)
        title = lines[0].strip() + " " + (lines[1].strip() if len(lines) > 1 else "")
        content = "\n".join(lines[2:]).strip() if len(lines) > 2 else ""

        sections.append({
            "section_id": len(sections) + 1,
            "title": title,
            "content": content,
            "combine": f"{title}\n{content}",
        })

    return sections

**SPLIT DOCUMENT BY ARTICLES (ĐIỀU)**

In [35]:
def split_articles_from_section(section_text, section_title=None):
    """
    Splits a section into a list of articles and appendices.
    Each article or appendix is a full text block.
    """
    # Match both Điều and PHỤ LỤC
    combined_pattern = re.compile(r"(Điều\s+\d+\..*|PHỤ LỤC\s+[IVXLCDM]+\b[^\n]*)", re.MULTILINE)

    matches = list(re.finditer(combined_pattern, section_text))
    if not matches:
        return [section_text.strip()]  # fallback: whole section

    articles = []
    for i in range(len(matches)):
        start = matches[i].start()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(section_text)
        chunk = section_text[start:end].strip()

        # Minimal cleaning but preserve newlines
        chunk = chunk.replace('\r\n', '\n').replace('\r', '\n')
        chunk = re.sub(r'[ \t]+', ' ', chunk)         # collapse multiple spaces
        chunk = re.sub(r'\n{3,}', '\n\n', chunk)      # collapse too many line breaks
        chunk = chunk.strip()

        articles.append(chunk)

    return articles

**CREATE ARTICLES DOCUMENT WITH GENERATE CONTEXT**

In [37]:
def create_article_list_with_section(
    api_key,
    document,
    document_name,
    document_doc_code,
    batch_size=3,
    delay_between_batches=5,
    max_retries=3,
    run_id=None
):
    """
    Create a table of article lists and its section with batching, delays, and checkpointing.
    Handles both Điều and PHỤ LỤC articles.
    """


    # Generate a unique ID for this processing run if not provided
    if not run_id:
        run_id = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Check for existing checkpoint
    checkpoint_data = load_checkpoint(run_id)
    if checkpoint_data:
        all_docs = checkpoint_data
        print(f"Resuming from checkpoint with {len(all_docs)} articles already processed")
    else:
        all_docs = []

    try:
        section_list = split_document_into_sections(document)

        total_sections = len(section_list)
        total_articles = 0
        for section in section_list:
            articles = split_articles_from_section(section["content"], section["title"])
            total_articles += len(articles)

        print(f"Starting processing: {total_articles} articles across {total_sections} sections")
        print(f"Already processed: {len(all_docs)} articles")

        d_index = 0   # Counter for Điều
        pl_index = 0  # Counter for PHỤ LỤC
        global_article_index = 0

        for section_idx, section in enumerate(section_list):
            section_content = section["content"]
            section_content_with_title = section["combine"]
            section_title = section["title"]
            section_id = section["section_id"]

            already_processed_articles = [
                doc for doc in all_docs if doc["section_id"] == section_id
            ]
            highest_processed = max([doc["article_index"] for doc in already_processed_articles], default=0)

            articles = split_articles_from_section(section_content, section_title)

            print(f"Processing section {section_idx+1}/{total_sections}: '{section_title}' - {len(articles)} articles")

            for i in range(0, len(articles), batch_size):
                batch_articles = articles[i:i+batch_size]

                for article_index, article_text in enumerate(batch_articles, start=i+1):
                    global_article_index += 1
                    if article_index <= highest_processed:
                        print(f"Skipping already processed article {article_index}")
                        continue

                    headline = extract_headline(article_text)
                    print(f"Processing article {article_index}/{len(articles)}: '{headline[:40]}...'")

                    # Decide prefix for reference ID
                    if headline.upper().startswith("PHỤ LỤC"):
                        pl_index += 1
                        reference_id = generate_reference_id(pl_index, document_doc_code, prefix="PL")
                        context_response = None
                        success = True
                    else:
                        d_index += 1
                        reference_id = generate_reference_id(d_index, document_doc_code, prefix="D")

                        # Generate context with retry logic
                        context_response = None
                        success = False
                        retry_count = 0

                        while not success and retry_count < max_retries:
                            try:
                                timeout = 30 + (retry_count * 30)
                                context_response = generate_context(
                                    api_key,
                                    section_content_with_title,
                                    article_text,
                                    timeout=timeout
                                )
                                success = True
                            except Exception as e:
                                retry_count += 1
                                wait_time = 5 * retry_count
                                print(f"Attempt {retry_count}/{max_retries} failed: {str(e)}")
                                print(f"Waiting {wait_time} seconds before retrying...")
                                time.sleep(wait_time)

                        if not success:
                            context_response = "Error: Context generation failed after multiple attempts"

                    article_data = {
                        "reference_id": reference_id,
                        "article_index": global_article_index,
                        "article_headline": headline,
                        "article_content": article_text,
                        "section_title": section_title,
                        "section_content": section_content_with_title,
                        "section_id": section_id,
                        "generated_context": context_response,
                        "processing_success": success,
                        "processing_timestamp": datetime.now().isoformat(),
                        "document_name": document_name
                    }

                    all_docs.append(article_data)

                    # Checkpoint saving
                    if len(all_docs) % CHECKPOINT_INTERVAL == 0:
                        save_checkpoint(all_docs, run_id)

                    time.sleep(random.uniform(0.5, 1.5))

                if i + batch_size < len(articles):
                    print(f"Completed batch. Waiting {delay_between_batches}s before next batch...")
                    time.sleep(delay_between_batches)

            save_checkpoint(all_docs, run_id)

    except Exception as e:
        print(f"❌ Error: {str(e)}")
        save_checkpoint(all_docs, f"{run_id}_emergency")
        raise

    save_checkpoint(all_docs, run_id)
    return pd.DataFrame(all_docs)


**FILTER ARTICLES LIST BY SECTION**

In [ ]:
def get_articles_by_section(df, section_id):
    """
    Filters articles in a specific section and combines context + article content.

    Args:
        df (pd.DataFrame): DataFrame with columns: section_id, generated_context, article_content.
        section_id (int): The section (Chương) to filter.

    Returns:
        List[str]: List of text inputs for embedding (context + content).
    """
    filtered_df = df[df["section_id"] == section_id].copy()
    filtered_df["embedding_input"] = filtered_df.apply(
        lambda row: f"{row['generated_context']}\n\n{row['article_content']}", axis=1
    )
    return filtered_df["embedding_input"].tolist(), filtered_df


#**GENERATE CONTEXT FOR EACH ARTICLE IN SECTION**

In [26]:
!pip install anthropic tenacity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 4.9 MB/s eta 0:00:00


In [27]:
import time
import random
from datetime import datetime
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
import anthropic

# Constants
CHECKPOINT_DIR = "/content/drive/MyDrive/COS30049 - PROJECT/checkpoints"
CHECKPOINT_INTERVAL = 10  # Save after every 10 articles

# Create checkpoint directory if it doesn't exist
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

In [28]:
#MODELS CHOICE
SONNET_37="claude-3-7-sonnet-20250219"
SONNET_35="claude-3-5-sonnet-20240620"
HAIKU="claude-3-5-haiku-20241022"

In [29]:
#PROMPT SET UP

DOCUMENT_CONTEXT_PROMPT="""
<document>
{section}
</document>
"""

CHUNK_CONTEXT_PROMPT="""
Đây là một phần của văn bản pháp lý (Điều luật) cần được tóm tắt để tối ưu hóa cho việc tìm kiếm ngữ nghĩa:

<chunk>
{article}
</chunk>

Hãy tạo một đoạn tóm tắt súc tích (40-60 từ) bao gồm:
1. Trích dẫn các từ khóa quan trọng nhất từ điều luật gốc
2. Mô tả ngắn gọn nội dung cốt lõi và phạm vi áp dụng
3. Sử dụng các từ đồng nghĩa/liên quan có giá trị ngữ nghĩa cao

Đảm bảo tóm tắt vừa bảo toàn ý nghĩa pháp lý chính xác, vừa chứa đủ "từ cầu nối" để kết nối hiệu quả với các phần khác của văn bản khi thực hiện tìm kiếm vector.
"""
SYSTEM_PROMPT="""
Bạn là một chuyên gia về luật và các thông tư bảo hiểm tại Việt Nam với hơn 20 năm kinh nghiệm trong ngành bảo hiểm.
"""
SYSTEM_PROMPT_EN=""""
You are an assistant that specializes in summarizing Vietnamese insurance policies from regulatory documents.
"""

In [30]:
# Add retry decorator to handle rate limits with exponential backoff
@retry(
    stop=stop_after_attempt(5),
    wait=wait_exponential(multiplier=1, min=2, max=60),
    retry=retry_if_exception_type((anthropic.RateLimitError, anthropic.APITimeoutError, anthropic.APIConnectionError))
)
def generate_context(api_key,doc: str, chunk: str, timeout=30) -> str:
    client = anthropic.Anthropic(api_key=api_key)
    normalized_doc = " ".join(doc.split())
    try:
        # Normalize the doc string to ensure consistent caching
        normalized_doc = re.sub(r'[ \t]+', ' ', doc)  # collapse spaces/tabs
        normalized_doc = re.sub(r'\n{3,}', '\n\n', normalized_doc)  # limit too many newlines
        normalized_doc = normalized_doc.strip()

        response = client.messages.create(
            model=HAIKU,
            max_tokens=1024,
            temperature=0.0,
            system=SYSTEM_PROMPT,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": DOCUMENT_CONTEXT_PROMPT.format(section=normalized_doc),
                            "cache_control": {"type": "ephemeral"}
                        },
                        {
                            "type": "text",
                            "text": CHUNK_CONTEXT_PROMPT.format(article=chunk),
                        }
                    ]
                }
            ],
            extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"},
            timeout=timeout  # Add timeout parameter
        )
        return response.content[0].text
    except Exception as e:
        print(f"Error in API call: {str(e)}")
        raise

def load_checkpoint(checkpoint_id):
    """Load the most recent checkpoint file"""
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f"checkpoint_{checkpoint_id}.json")
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, 'r') as f:
            return json.load(f)
    return None

def save_checkpoint(all_docs, checkpoint_id):
    """Save current progress to a checkpoint file"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f"checkpoint_{checkpoint_id}.json")
    with open(checkpoint_path, 'w') as f:
        json.dump(all_docs, f)
    print(f"Checkpoint saved: {checkpoint_path}")

#**EMBEDDING CONTENT**

**SETUP INSTALL AND IMPORT**

In [5]:
from sentence_transformers import SentenceTransformer
import torch
import numpy as np

##**DENSE VECTOR EMBEDDING WITH HUGGINGFACE MODEL**

In [5]:
def huggingface_embedding(content, model)-> np.ndarray:
  model = SentenceTransformer(model, trust_remote_code=True)
  embeddings = model.encode(
    content,
    batch_size=4,
    convert_to_numpy=True,
    show_progress_bar=True,
    device='cuda' if torch.cuda.is_available() else 'cpu'
    # device='cpu'
  )
  # Output: embeddings là numpy array chứa vector embedding của tất cả văn bản, shape là (số văn bản, 76/81024)
  return embeddings

##**SPARSE VECTOR EMBEDDING USING SKIKITLEARN TF_IDF VECTORIZER**

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from typing import List, Tuple, Dict
import scipy.sparse

def generate_sparse_vectors(texts: List[str]) -> Tuple[TfidfVectorizer, List[Dict[str, List[float]]]]:
    """
    Generate sparse vectors from a list of text strings using TfidfVectorizer.

    Args:
        texts (List[str]): List of input strings (e.g., embedding_input)

    Returns:
        vectorizer (TfidfVectorizer): The fitted TF-IDF vectorizer
        sparse_vectors (List[Dict]): List of dicts with 'indices' and 'values' per input text
    """
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)

    def tfidf_to_sparse_dict(row: scipy.sparse.csr_matrix) -> Dict[str, List[float]]:
        coo = row.tocoo()
        return {
            "indices": coo.col.tolist(),
            "values": coo.data.tolist()
        }

    sparse_vectors = [tfidf_to_sparse_dict(tfidf_matrix[i]) for i in range(tfidf_matrix.shape[0])]
    return vectorizer, sparse_vectors


#**MAIN DOCUMENTS PROCESS FLOW**

##**PROCESS DOCUMENT THÔNG TƯ 67**

###**LOAD FUNCTION**

####**LOAD PDF (START FROM THIS IF DOESN'T SAVED PROGRESS YET)**

In [ ]:
d_67 = load_pdf("/content/drive/MyDrive/COS30049 - PROJECT/Dataset/thong tu/67_2023_TT-BTC_m_548480.pdf")

####**LOAD PARQUET (LOADED SAVED PROGRESS)**

In [7]:
df_loaded = pd.read_parquet("/content/drive/MyDrive/COS30049 - PROJECT/Dataset/d_67_articles_with_embedding.parquet")

In [ ]:
display(df_loaded.head())

,reference_id,Unnamed: 0,article_index,article_headline,article_content,section_title,section_content,section_id,generated_context,processing_success,processing_timestamp,usage_count,document_name,embedding,article_index_local,embedding_length
0,TT_67_2023_BTC_D1,0,1,Điều 1. Phạm vi điều chỉnh,Điều 1. Phạm vi điều chỉnh\n1. Thông tư này qu...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nThông tư hướng dẫn chi tiết các qu...,True,2025-03-30T07:34:10.672326,1,Thông tư 67,"[0.014020857, -0.032874405, 0.019974034, 0.066...",1,1024
1,TT_67_2023_BTC_D2,1,2,Điều 2. Đối tượng áp dụng,Điều 2. Đối tượng áp dụng\n1. Doanh nghiệp bảo...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nĐiều luật quy định đối tượng áp dụ...,True,2025-03-30T07:34:14.249133,1,Thông tư 67,"[-0.006178328, -0.05860321, 0.0059361183, 0.05...",2,1024
2,TT_67_2023_BTC_D3,2,3,Điều 3. Cung cấp và cập nhật thông tin,Điều 3. Cung cấp và cập nhật thông tin\n1. Thô...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nĐiều 3 quy định chi tiết việc cung...,True,2025-03-30T07:34:18.215150,1,Thông tư 67,"[-0.003096825, -0.04325807, 0.010223848, 0.066...",3,1024
3,TT_67_2023_BTC_D4,3,4,"Điều 4. Cung cấp dịch vụ, sản phẩm bảo hiểm tr...","Điều 4. Cung cấp dịch vụ, sản phẩm bảo hiểm tr...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...",2,Tóm tắt:\n\nQuy định về cung cấp dịch vụ bảo h...,True,2025-03-30T07:34:24.557895,1,Thông tư 67,"[0.03668985, -0.12909615, 0.010524653, 0.00540...",1,1024
4,TT_67_2023_BTC_D5,4,5,"Điều 5. Hình thức cung cấp dịch vụ, sản phẩm b...","Điều 5. Hình thức cung cấp dịch vụ, sản phẩm b...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...",2,Tóm tắt:\n\nQuy định các hình thức cung cấp dị...,True,2025-03-30T07:34:29.222356,1,Thông tư 67,"[0.042817216, -0.13181126, 0.020213768, -0.006...",2,1024


###**GENERATE ARTICLES LIST WITH GENERATED CONTEXT FROM LLM**

In [ ]:
d_67_articles = create_article_list_with_section(api_key=ANTHROPIC_API_KEY,document=d_67)

Starting processing: 62 articles across 7 sections
Already processed: 0 articles
Processing section 1/7: 'Chương I QUY ĐỊNH CHUNG' - 3 articles
Processing article 1/3: 'Điều 1. Phạm vi điều chỉnh...'
Processing article 2/3: 'Điều 2. Đối tượng áp dụng...'
Processing article 3/3: 'Điều 3. Cung cấp và cập nhật thông tin...'
Checkpoint saved: /content/drive/MyDrive/COS30049 - PROJECT/checkpoints/checkpoint_20250330_073406.json
Processing section 2/7: 'Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM TRÊN MÔI TRƯỜNG MẠNG' - 5 articles
Processing article 1/5: 'Điều 4. Cung cấp dịch vụ, sản phẩm bảo h...'
Processing article 2/5: 'Điều 5. Hình thức cung cấp dịch vụ, sản ...'
Processing article 3/5: 'Điều 6. Thông báo về việc cung cấp dịch ...'
Completed batch. Progress: 6/62 articles processed
Waiting 5 seconds before next batch...
Processing article 4/5: 'Điều 7. Quy định về dịch vụ, kỹ thuật, b...'
Processing article 5/5: 'Điều 8. Trách nhiệm của doanh nghiệp bảo...'
Checkpoint saved: /content/

###**GENERATE EMBEDDING USING HUGGINGFACE MODELS**

In [13]:
#SETUP MODEL
VIETNAMESE_DOCUMENT_EMBEDDING_MODEL = 'dangvantuan/vietnamese-document-embedding'

In [ ]:
#PREPARE DATA
d_67_articles_with_embedding_hugginface = df_loaded.copy()
d_67_articles_with_embedding_hugginface["embedding_input"] = d_67_articles_with_embedding_hugginface.apply(
        lambda row: f"{row['generated_context']}\n\n{row['article_content']}", axis=1
    )
d_67_articles_with_embedding_hugginface["embedding_input"].tolist()

In [16]:
#CHECKING DATA BEFOR EMBEDDING
d_67_articles_with_embedding_hugginface.head()

,reference_id,Unnamed: 0,article_index,article_headline,article_content,section_title,section_content,section_id,generated_context,processing_success,processing_timestamp,usage_count,document_name,embedding,article_index_local,embedding_length,embedding_input
0,TT_67_2023_BTC_D1,0,1,Điều 1. Phạm vi điều chỉnh,Điều 1. Phạm vi điều chỉnh\n1. Thông tư này qu...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nThông tư hướng dẫn chi tiết các qu...,True,2025-03-30T07:34:10.672326,1,Thông tư 67,"[0.014020857, -0.032874405, 0.019974034, 0.066...",1,1024,Tóm tắt:\n\nThông tư hướng dẫn chi tiết các qu...
1,TT_67_2023_BTC_D2,1,2,Điều 2. Đối tượng áp dụng,Điều 2. Đối tượng áp dụng\n1. Doanh nghiệp bảo...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nĐiều luật quy định đối tượng áp dụ...,True,2025-03-30T07:34:14.249133,1,Thông tư 67,"[-0.006178328, -0.05860321, 0.0059361183, 0.05...",2,1024,Tóm tắt:\n\nĐiều luật quy định đối tượng áp dụ...
2,TT_67_2023_BTC_D3,2,3,Điều 3. Cung cấp và cập nhật thông tin,Điều 3. Cung cấp và cập nhật thông tin\n1. Thô...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nĐiều 3 quy định chi tiết việc cung...,True,2025-03-30T07:34:18.215150,1,Thông tư 67,"[-0.003096825, -0.04325807, 0.010223848, 0.066...",3,1024,Tóm tắt:\n\nĐiều 3 quy định chi tiết việc cung...
3,TT_67_2023_BTC_D4,3,4,"Điều 4. Cung cấp dịch vụ, sản phẩm bảo hiểm tr...","Điều 4. Cung cấp dịch vụ, sản phẩm bảo hiểm tr...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...",2,Tóm tắt:\n\nQuy định về cung cấp dịch vụ bảo h...,True,2025-03-30T07:34:24.557895,1,Thông tư 67,"[0.03668985, -0.12909615, 0.010524653, 0.00540...",1,1024,Tóm tắt:\n\nQuy định về cung cấp dịch vụ bảo h...
4,TT_67_2023_BTC_D5,4,5,"Điều 5. Hình thức cung cấp dịch vụ, sản phẩm b...","Điều 5. Hình thức cung cấp dịch vụ, sản phẩm b...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...",2,Tóm tắt:\n\nQuy định các hình thức cung cấp dị...,True,2025-03-30T07:34:29.222356,1,Thông tư 67,"[0.042817216, -0.13181126, 0.020213768, -0.006...",2,1024,Tóm tắt:\n\nQuy định các hình thức cung cấp dị...


In [17]:
#EMBEDDING
embeddings = huggingface_embedding(d_67_articles_with_embedding_hugginface['embedding_input'], VIETNAMESE_DOCUMENT_EMBEDDING_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [18]:
#Check Embedding Vector Dimension
embeddings.shape

(62, 768)

In [20]:
#ADD EMBEDDING VECTOR BACK TO TABLE'
d_67_articles_with_embedding_hugginface['vietnamese_document_embedding'] = embeddings.tolist()

In [21]:
#FINAL CHECKING
d_67_articles_with_embedding_hugginface.head()

,reference_id,Unnamed: 0,article_index,article_headline,article_content,section_title,section_content,section_id,generated_context,processing_success,processing_timestamp,usage_count,document_name,embedding,article_index_local,embedding_length,embedding_input,vietnamese_document_embedding
0,TT_67_2023_BTC_D1,0,1,Điều 1. Phạm vi điều chỉnh,Điều 1. Phạm vi điều chỉnh\n1. Thông tư này qu...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nThông tư hướng dẫn chi tiết các qu...,True,2025-03-30T07:34:10.672326,1,Thông tư 67,"[0.014020857, -0.032874405, 0.019974034, 0.066...",1,1024,Tóm tắt:\n\nThông tư hướng dẫn chi tiết các qu...,"[-0.015283266082406044, -0.013573165982961655,..."
1,TT_67_2023_BTC_D2,1,2,Điều 2. Đối tượng áp dụng,Điều 2. Đối tượng áp dụng\n1. Doanh nghiệp bảo...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nĐiều luật quy định đối tượng áp dụ...,True,2025-03-30T07:34:14.249133,1,Thông tư 67,"[-0.006178328, -0.05860321, 0.0059361183, 0.05...",2,1024,Tóm tắt:\n\nĐiều luật quy định đối tượng áp dụ...,"[-0.06707688421010971, -0.033460795879364014, ..."
2,TT_67_2023_BTC_D3,2,3,Điều 3. Cung cấp và cập nhật thông tin,Điều 3. Cung cấp và cập nhật thông tin\n1. Thô...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nĐiều 3 quy định chi tiết việc cung...,True,2025-03-30T07:34:18.215150,1,Thông tư 67,"[-0.003096825, -0.04325807, 0.010223848, 0.066...",3,1024,Tóm tắt:\n\nĐiều 3 quy định chi tiết việc cung...,"[-0.018949996680021286, 0.00022413932310882956..."
3,TT_67_2023_BTC_D4,3,4,"Điều 4. Cung cấp dịch vụ, sản phẩm bảo hiểm tr...","Điều 4. Cung cấp dịch vụ, sản phẩm bảo hiểm tr...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...",2,Tóm tắt:\n\nQuy định về cung cấp dịch vụ bảo h...,True,2025-03-30T07:34:24.557895,1,Thông tư 67,"[0.03668985, -0.12909615, 0.010524653, 0.00540...",1,1024,Tóm tắt:\n\nQuy định về cung cấp dịch vụ bảo h...,"[-0.060357715934515, -0.032350294291973114, -0..."
4,TT_67_2023_BTC_D5,4,5,"Điều 5. Hình thức cung cấp dịch vụ, sản phẩm b...","Điều 5. Hình thức cung cấp dịch vụ, sản phẩm b...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...",2,Tóm tắt:\n\nQuy định các hình thức cung cấp dị...,True,2025-03-30T07:34:29.222356,1,Thông tư 67,"[0.042817216, -0.13181126, 0.020213768, -0.006...",2,1024,Tóm tắt:\n\nQuy định các hình thức cung cấp dị...,"[-0.09163884073495865, -0.056286752223968506, ..."


###**SAVE TO PARQUET/CSV**

#####**SAVE TO PARQUET (SAFEST OPTION)**

In [ ]:
def save_document_to_parquet(document, path):
  """
  ARGS:
  document: document to save
  path: path to save document

  Create a copy of a Document then save to parquet file
  """
  saved_document = document.copy()
  saved_document.to_parquet(path)

In [ ]:
#RECHECK BEFORE SAVE
display(d_67_articles_with_embedding_hugginface.head())

,reference_id,Unnamed: 0,article_index,article_headline,article_content,section_title,section_content,section_id,generated_context,processing_success,processing_timestamp,usage_count,document_name,embedding,article_index_local,embedding_length,embedding_input,vietnamese_document_embedding
0,TT_67_2023_BTC_D1,0,1,Điều 1. Phạm vi điều chỉnh,Điều 1. Phạm vi điều chỉnh\n1. Thông tư này qu...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nThông tư hướng dẫn chi tiết các qu...,True,2025-03-30T07:34:10.672326,1,Thông tư 67,"[0.014020857, -0.032874405, 0.019974034, 0.066...",1,1024,Tóm tắt:\n\nThông tư hướng dẫn chi tiết các qu...,"[-0.015283266082406044, -0.013573165982961655,..."
1,TT_67_2023_BTC_D2,1,2,Điều 2. Đối tượng áp dụng,Điều 2. Đối tượng áp dụng\n1. Doanh nghiệp bảo...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nĐiều luật quy định đối tượng áp dụ...,True,2025-03-30T07:34:14.249133,1,Thông tư 67,"[-0.006178328, -0.05860321, 0.0059361183, 0.05...",2,1024,Tóm tắt:\n\nĐiều luật quy định đối tượng áp dụ...,"[-0.06707688421010971, -0.033460795879364014, ..."
2,TT_67_2023_BTC_D3,2,3,Điều 3. Cung cấp và cập nhật thông tin,Điều 3. Cung cấp và cập nhật thông tin\n1. Thô...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nĐiều 3 quy định chi tiết việc cung...,True,2025-03-30T07:34:18.215150,1,Thông tư 67,"[-0.003096825, -0.04325807, 0.010223848, 0.066...",3,1024,Tóm tắt:\n\nĐiều 3 quy định chi tiết việc cung...,"[-0.018949996680021286, 0.00022413932310882956..."
3,TT_67_2023_BTC_D4,3,4,"Điều 4. Cung cấp dịch vụ, sản phẩm bảo hiểm tr...","Điều 4. Cung cấp dịch vụ, sản phẩm bảo hiểm tr...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...",2,Tóm tắt:\n\nQuy định về cung cấp dịch vụ bảo h...,True,2025-03-30T07:34:24.557895,1,Thông tư 67,"[0.03668985, -0.12909615, 0.010524653, 0.00540...",1,1024,Tóm tắt:\n\nQuy định về cung cấp dịch vụ bảo h...,"[-0.060357715934515, -0.032350294291973114, -0..."
4,TT_67_2023_BTC_D5,4,5,"Điều 5. Hình thức cung cấp dịch vụ, sản phẩm b...","Điều 5. Hình thức cung cấp dịch vụ, sản phẩm b...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...","Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...",2,Tóm tắt:\n\nQuy định các hình thức cung cấp dị...,True,2025-03-30T07:34:29.222356,1,Thông tư 67,"[0.042817216, -0.13181126, 0.020213768, -0.006...",2,1024,Tóm tắt:\n\nQuy định các hình thức cung cấp dị...,"[-0.09163884073495865, -0.056286752223968506, ..."


In [ ]:
save_document_to_parquet(d_67_articles_with_embedding_hugginface, "/content/drive/MyDrive/COS30049 - PROJECT/Dataset/d_67_articles_with_embedding_hugginface.parquet")

##**PROCESS DOCUMENT 50 (THÔNG TƯ 50)**

In [6]:
#LOAD PARQUET
df_loaded = pd.read_parquet("/content/drive/MyDrive/COS30049 - PROJECT/Dataset/d_50_articles.parquet")

In [100]:
#LOAD PDF
d_50 = load_pdf("/content/drive/MyDrive/COS30049 - PROJECT/Dataset/thong tu/TT50.pdf")

In [101]:
#SPLIT DOCUMENT INTO ARTCLES AND GENERATE CONTEXT FOR EACH ARTICLE
document_50_name = "Thông tư 50"
document_50_doc_code = "TT_50_2022_BTC"
d_50_articles = create_article_list_with_section(api_key=ANTHROPIC_API_KEY,document=d_50,document_name=document_50_name,document_doc_code=document_50_doc_code)

Starting processing: 45 articles across 3 sections
Already processed: 0 articles
Processing section 1/3: 'Chương I QUY ĐỊNH CHUNG' - 6 articles
Processing article 1/6: 'Điều 1. Phạm vi điều chỉnh...'
Processing article 2/6: 'Điều 2. Đối tượng áp dụng...'
Processing article 3/6: 'Điều 3. Giải thích từ ngữ...'
Completed batch. Waiting 5s before next batch...
Processing article 4/6: 'Điều 4. Giấy chứng nhận bảo hiểm bắt buộ...'
Processing article 5/6: 'Điều 5. Loại trừ trách nhiệm bảo hiểm ch...'
Processing article 6/6: 'Điều 6. Chấm dứt hợp đồng bảo hiểm...'
Checkpoint saved: /content/drive/MyDrive/COS30049 - PROJECT/checkpoints/checkpoint_20250331_094502.json
Processing section 2/3: 'Chương II QUY ĐỊNH CỤ THỂ' - 30 articles
Processing article 1/30: 'Điều 7. Đối tượng bảo hiểm và số tiền bả...'
Processing article 2/30: 'Điều 8. Phạm vi bảo hiểm, loại trừ trách...'
Processing article 3/30: 'Điều 9. Thời hạn bảo hiểm...'
Completed batch. Waiting 5s before next batch...
Processing article 4

In [103]:
d_50_articles.tail()

,reference_id,article_index,article_headline,article_content,section_title,section_content,section_id,generated_context,processing_success,processing_timestamp,document_name
40,TT_50_2022_BTC_PL4,41,PHỤ LỤC IV,PHỤ LỤC IV\nBẢNG TỶ LỆ TRẢ TIỀN BỒI THƯỜNG BẢO...,Chương III ĐIỀU KHOẢN THI HÀNH,Chương III ĐIỀU KHOẢN THI HÀNH\nĐiều 37. Điều ...,3,None,True,2025-03-31T10:01:58.455365,Thông tư 50
41,TT_50_2022_BTC_PL5,42,PHỤ LỤC V,PHỤ LỤC V\nBẢNG QUY ĐỊNH TRẢ TIỀN BỒI THƯỜNG T...,Chương III ĐIỀU KHOẢN THI HÀNH,Chương III ĐIỀU KHOẢN THI HÀNH\nĐiều 37. Điều ...,3,None,True,2025-03-31T10:01:59.086649,Thông tư 50
42,TT_50_2022_BTC_PL6,43,PHỤ LỤC VI,PHỤ LỤC VI\nBÁO CÁO DOANH THU PHÍ BẢO HIỂM BẮT...,Chương III ĐIỀU KHOẢN THI HÀNH,Chương III ĐIỀU KHOẢN THI HÀNH\nĐiều 37. Điều ...,3,None,True,2025-03-31T10:02:05.257346,Thông tư 50
43,TT_50_2022_BTC_PL7,44,PHỤ LỤC VII,PHỤ LỤC VII\nBÁO CÁO BỒI THƯỜNG BẢO HIỂM BẮT B...,Chương III ĐIỀU KHOẢN THI HÀNH,Chương III ĐIỀU KHOẢN THI HÀNH\nĐiều 37. Điều ...,3,None,True,2025-03-31T10:02:05.890397,Thông tư 50
44,TT_50_2022_BTC_PL8,45,PHỤ LỤC VIII,PHỤ LỤC VIII\nBÁO CÁO KẾT QUẢ KINH DOANH BẢO H...,Chương III ĐIỀU KHOẢN THI HÀNH,Chương III ĐIỀU KHOẢN THI HÀNH\nĐiều 37. Điều ...,3,None,True,2025-03-31T10:02:06.841414,Thông tư 50


###**GENERATE EMBEDDING USING HUGGINGFACE MODELS**

In [7]:
#SETUP MODEL
VIETNAMESE_DOCUMENT_EMBEDDING_MODEL = 'dangvantuan/vietnamese-document-embedding'
#PREPARE DATA
d_50_articles_with_embedding_hugginface = df_loaded.copy()
d_50_articles_with_embedding_hugginface["embedding_input"] = d_50_articles_with_embedding_hugginface.apply(
        lambda row: f"{row['generated_context']}\n\n{row['article_content']}", axis=1
    )
d_50_articles_with_embedding_hugginface["embedding_input"].tolist()
#CHECKING DATA BEFOR EMBEDDING
d_50_articles_with_embedding_hugginface.tail()

,reference_id,article_index,article_headline,article_content,section_title,section_content,section_id,generated_context,processing_success,processing_timestamp,document_name,embedding_input
40,TT_50_2022_BTC_PL4,41,PHỤ LỤC IV,PHỤ LỤC IV\nBẢNG TỶ LỆ TRẢ TIỀN BỒI THƯỜNG BẢO...,Chương III ĐIỀU KHOẢN THI HÀNH,Chương III ĐIỀU KHOẢN THI HÀNH\nĐiều 37. Điều ...,3,None,True,2025-03-31T10:01:58.455365,Thông tư 50,None\n\nPHỤ LỤC IV\nBẢNG TỶ LỆ TRẢ TIỀN BỒI TH...
41,TT_50_2022_BTC_PL5,42,PHỤ LỤC V,PHỤ LỤC V\nBẢNG QUY ĐỊNH TRẢ TIỀN BỒI THƯỜNG T...,Chương III ĐIỀU KHOẢN THI HÀNH,Chương III ĐIỀU KHOẢN THI HÀNH\nĐiều 37. Điều ...,3,None,True,2025-03-31T10:01:59.086649,Thông tư 50,None\n\nPHỤ LỤC V\nBẢNG QUY ĐỊNH TRẢ TIỀN BỒI ...
42,TT_50_2022_BTC_PL6,43,PHỤ LỤC VI,PHỤ LỤC VI\nBÁO CÁO DOANH THU PHÍ BẢO HIỂM BẮT...,Chương III ĐIỀU KHOẢN THI HÀNH,Chương III ĐIỀU KHOẢN THI HÀNH\nĐiều 37. Điều ...,3,None,True,2025-03-31T10:02:05.257346,Thông tư 50,None\n\nPHỤ LỤC VI\nBÁO CÁO DOANH THU PHÍ BẢO ...
43,TT_50_2022_BTC_PL7,44,PHỤ LỤC VII,PHỤ LỤC VII\nBÁO CÁO BỒI THƯỜNG BẢO HIỂM BẮT B...,Chương III ĐIỀU KHOẢN THI HÀNH,Chương III ĐIỀU KHOẢN THI HÀNH\nĐiều 37. Điều ...,3,None,True,2025-03-31T10:02:05.890397,Thông tư 50,None\n\nPHỤ LỤC VII\nBÁO CÁO BỒI THƯỜNG BẢO HI...
44,TT_50_2022_BTC_PL8,45,PHỤ LỤC VIII,PHỤ LỤC VIII\nBÁO CÁO KẾT QUẢ KINH DOANH BẢO H...,Chương III ĐIỀU KHOẢN THI HÀNH,Chương III ĐIỀU KHOẢN THI HÀNH\nĐiều 37. Điều ...,3,None,True,2025-03-31T10:02:06.841414,Thông tư 50,None\n\nPHỤ LỤC VIII\nBÁO CÁO KẾT QUẢ KINH DOA...


In [8]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [11]:
#EMBEDDING
embeddings = huggingface_embedding(d_50_articles_with_embedding_hugginface['embedding_input'], VIETNAMESE_DOCUMENT_EMBEDDING_MODEL)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

In [12]:
#Check Embedding Vector Dimension
embeddings.shape

(45, 768)

In [14]:
#ADD EMBEDDING VECTOR BACK TO TABLE'
d_50_articles_with_embedding_hugginface['vietnamese_document_embedding'] = embeddings.tolist()

In [15]:
#FINAL CHECKING
d_50_articles_with_embedding_hugginface.head()

,reference_id,article_index,article_headline,article_content,section_title,section_content,section_id,generated_context,processing_success,processing_timestamp,document_name,embedding_input,vietnamese_document_embedding
0,TT_50_2022_BTC_D1,1,Điều 1. Phạm vi điều chỉnh,Điều 1. Phạm vi điều chỉnh\nThông tư này hướng...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nThông tư quy định chi tiết về các ...,True,2025-03-31T09:45:05.356299,Thông tư 50,Tóm tắt:\n\nThông tư quy định chi tiết về các ...,"[-0.07406723499298096, -0.08412662893533707, -..."
1,TT_50_2022_BTC_D2,2,Điều 2. Đối tượng áp dụng,Điều 2. Đối tượng áp dụng\nThông tư này áp dụn...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nThông tư quy định phạm vi áp dụng ...,True,2025-03-31T09:45:10.663154,Thông tư 50,Tóm tắt:\n\nThông tư quy định phạm vi áp dụng ...,"[-0.0666598379611969, -0.08404409140348434, -0..."
2,TT_50_2022_BTC_D3,3,Điều 3. Giải thích từ ngữ,"Điều 3. Giải thích từ ngữ\nTrong Thông tư này,...",Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt Điều 3 - Giải thích từ ngữ:\n\nĐiều kh...,True,2025-03-31T09:45:15.202319,Thông tư 50,Tóm tắt Điều 3 - Giải thích từ ngữ:\n\nĐiều kh...,"[-0.03349874168634415, -0.046798840165138245, ..."
3,TT_50_2022_BTC_D4,4,Điều 4. Giấy chứng nhận bảo hiểm bắt buộc tron...,Điều 4. Giấy chứng nhận bảo hiểm bắt buộc tron...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nGiấy chứng nhận bảo hiểm bắt buộc ...,True,2025-03-31T09:45:24.235523,Thông tư 50,Tóm tắt:\n\nGiấy chứng nhận bảo hiểm bắt buộc ...,"[-0.05826373025774956, -0.05339961126446724, -..."
4,TT_50_2022_BTC_D5,5,Điều 5. Loại trừ trách nhiệm bảo hiểm chung,Điều 5. Loại trừ trách nhiệm bảo hiểm chung\nD...,Chương I QUY ĐỊNH CHUNG,Chương I QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều ...,1,Tóm tắt:\n\nĐiều luật quy định các trường hợp ...,True,2025-03-31T09:45:28.441019,Thông tư 50,Tóm tắt:\n\nĐiều luật quy định các trường hợp ...,"[-0.047347571700811386, -0.02445059083402157, ..."


###**SAVE DATA**

In [16]:
saved_data = d_50_articles_with_embedding_hugginface.copy()
saved_data.to_parquet("/content/drive/MyDrive/COS30049 - PROJECT/Dataset/d_50_articles.parquet")

##**PROCESS LUẬT KINH DOANH BẢO HIỂM (L_08_2022_QH)**

In [ ]:
#LOAD PARQUET (IF ALREADY HAVE SAVED PROGRESS)
df_loaded = pd.read_parquet("/content/drive/MyDrive/COS30049 - PROJECT/Dataset/luat-kdbh.parquet")

In [38]:
#LOAD PDF
loaded_document = load_pdf("/content/drive/MyDrive/COS30049 - PROJECT/Dataset/thong tu/08_2022_QH15_m_465916.pdf")

In [40]:
#SPLIT DOCUMENT INTO ARTCLES AND GENERATE CONTEXT FOR EACH ARTICLE
document_name = "Luật Kinh Doanh Bảo Hiểm"
document_doc_code = "L_08_2022_QH_15"
document_articles = create_article_list_with_section(api_key=ANTHROPIC_API_KEY,document=loaded_document,document_name=document_name,document_doc_code=document_doc_code)

Starting processing: 157 articles across 7 sections
Already processed: 0 articles
Processing section 1/7: 'Chương I NHỮNG QUY ĐỊNH CHUNG' - 14 articles
Processing article 1/14: 'Điều 1. Phạm vi điều chỉnh...'
Processing article 2/14: 'Điều 2. Đối tượng áp dụng...'
Processing article 3/14: 'Điều 3. Áp dụng Luật Kinh doanh bảo hiểm...'
Completed batch. Waiting 5s before next batch...
Processing article 4/14: 'Điều 4. Giải thích từ ngữ...'
Processing article 5/14: 'Điều 5. Chính sách phát triển hoạt động ...'
Processing article 6/14: 'Điều 6. Nguyên tắc cung cấp và sử dụng d...'
Completed batch. Waiting 5s before next batch...
Processing article 7/14: 'Điều 7. Các loại hình bảo hiểm...'
Processing article 8/14: 'Điều 8. Bảo hiểm bắt buộc...'
Processing article 9/14: 'Điều 9. Các hành vi bị nghiêm cấm...'
Completed batch. Waiting 5s before next batch...
Processing article 10/14: 'Điều 10. Tổ chức xã hội - nghề nghiệp tr...'
Checkpoint saved: /content/drive/MyDrive/COS30049 - PROJECT/checkp

In [43]:
document_articles.tail()

,reference_id,article_index,article_headline,article_content,section_title,section_content,section_id,generated_context,processing_success,processing_timestamp,document_name
152,L_08_2022_QH_15_D153,153,Điều 153. Quyền yêu cầu cung cấp thông tin của...,Điều 153. Quyền yêu cầu cung cấp thông tin của...,Chương VI QUẢN LÝ NHÀ NƯỚC VỀ HOẠT ĐỘNG KINH D...,Chương VI QUẢN LÝ NHÀ NƯỚC VỀ HOẠT ĐỘNG KINH D...,6,Tóm tắt:\n\nĐiều 153 quy định quyền hạn của Bộ...,True,2025-03-31T12:39:58.770351,Luật Kinh Doanh Bảo Hiểm
153,L_08_2022_QH_15_D154,154,Điều 154. Thanh tra hoạt động kinh doanh bảo hiểm,Điều 154. Thanh tra hoạt động kinh doanh bảo h...,Chương VI QUẢN LÝ NHÀ NƯỚC VỀ HOẠT ĐỘNG KINH D...,Chương VI QUẢN LÝ NHÀ NƯỚC VỀ HOẠT ĐỘNG KINH D...,6,Tóm tắt:\n\nĐiều 154 quy định về thanh tra chu...,True,2025-03-31T12:40:10.212626,Luật Kinh Doanh Bảo Hiểm
154,L_08_2022_QH_15_D155,155,"Điều 155. Sửa đổi, bổ sung Phụ lục IV của Luật...","Điều 155. Sửa đổi, bổ sung Phụ lục IV của Luật...",Chương VII ĐIỀU KHOẢN THI HÀNH,Chương VII ĐIỀU KHOẢN THI HÀNH\nĐiều 155. Sửa ...,7,Tóm tắt:\n\nĐiều luật bổ sung hai ngành nghề đ...,True,2025-03-31T12:40:14.731740,Luật Kinh Doanh Bảo Hiểm
155,L_08_2022_QH_15_D156,156,Điều 156. Hiệu lực thi hành,Điều 156. Hiệu lực thi hành\n1. Luật này có hi...,Chương VII ĐIỀU KHOẢN THI HÀNH,Chương VII ĐIỀU KHOẢN THI HÀNH\nĐiều 155. Sửa ...,7,Tóm tắt:\n\nĐiều 156 quy định hiệu lực thi hàn...,True,2025-03-31T12:40:19.133501,Luật Kinh Doanh Bảo Hiểm
156,L_08_2022_QH_15_D157,157,Điều 157. Điều khoản chuyển tiếp,Điều 157. Điều khoản chuyển tiếp\n1. Các hợp đ...,Chương VII ĐIỀU KHOẢN THI HÀNH,Chương VII ĐIỀU KHOẢN THI HÀNH\nĐiều 155. Sửa ...,7,Tóm tắt Điều 157 - Điều khoản chuyển tiếp:\n\n...,True,2025-03-31T12:40:24.222583,Luật Kinh Doanh Bảo Hiểm


In [44]:
#SETUP MODEL
VIETNAMESE_DOCUMENT_EMBEDDING_MODEL = 'dangvantuan/vietnamese-document-embedding'
#PREPARE DATA
luat_kd_bao_hiem_document_embedding_huggingface = document_articles.copy()
luat_kd_bao_hiem_document_embedding_huggingface["embedding_input"] = luat_kd_bao_hiem_document_embedding_huggingface.apply(
        lambda row: f"{row['generated_context']}\n\n{row['article_content']}", axis=1
    )
luat_kd_bao_hiem_document_embedding_huggingface["embedding_input"].tolist()
#CHECKING DATA BEFORE EMBEDDING
luat_kd_bao_hiem_document_embedding_huggingface.tail()

,reference_id,article_index,article_headline,article_content,section_title,section_content,section_id,generated_context,processing_success,processing_timestamp,document_name,embedding_input
152,L_08_2022_QH_15_D153,153,Điều 153. Quyền yêu cầu cung cấp thông tin của...,Điều 153. Quyền yêu cầu cung cấp thông tin của...,Chương VI QUẢN LÝ NHÀ NƯỚC VỀ HOẠT ĐỘNG KINH D...,Chương VI QUẢN LÝ NHÀ NƯỚC VỀ HOẠT ĐỘNG KINH D...,6,Tóm tắt:\n\nĐiều 153 quy định quyền hạn của Bộ...,True,2025-03-31T12:39:58.770351,Luật Kinh Doanh Bảo Hiểm,Tóm tắt:\n\nĐiều 153 quy định quyền hạn của Bộ...
153,L_08_2022_QH_15_D154,154,Điều 154. Thanh tra hoạt động kinh doanh bảo hiểm,Điều 154. Thanh tra hoạt động kinh doanh bảo h...,Chương VI QUẢN LÝ NHÀ NƯỚC VỀ HOẠT ĐỘNG KINH D...,Chương VI QUẢN LÝ NHÀ NƯỚC VỀ HOẠT ĐỘNG KINH D...,6,Tóm tắt:\n\nĐiều 154 quy định về thanh tra chu...,True,2025-03-31T12:40:10.212626,Luật Kinh Doanh Bảo Hiểm,Tóm tắt:\n\nĐiều 154 quy định về thanh tra chu...
154,L_08_2022_QH_15_D155,155,"Điều 155. Sửa đổi, bổ sung Phụ lục IV của Luật...","Điều 155. Sửa đổi, bổ sung Phụ lục IV của Luật...",Chương VII ĐIỀU KHOẢN THI HÀNH,Chương VII ĐIỀU KHOẢN THI HÀNH\nĐiều 155. Sửa ...,7,Tóm tắt:\n\nĐiều luật bổ sung hai ngành nghề đ...,True,2025-03-31T12:40:14.731740,Luật Kinh Doanh Bảo Hiểm,Tóm tắt:\n\nĐiều luật bổ sung hai ngành nghề đ...
155,L_08_2022_QH_15_D156,156,Điều 156. Hiệu lực thi hành,Điều 156. Hiệu lực thi hành\n1. Luật này có hi...,Chương VII ĐIỀU KHOẢN THI HÀNH,Chương VII ĐIỀU KHOẢN THI HÀNH\nĐiều 155. Sửa ...,7,Tóm tắt:\n\nĐiều 156 quy định hiệu lực thi hàn...,True,2025-03-31T12:40:19.133501,Luật Kinh Doanh Bảo Hiểm,Tóm tắt:\n\nĐiều 156 quy định hiệu lực thi hàn...
156,L_08_2022_QH_15_D157,157,Điều 157. Điều khoản chuyển tiếp,Điều 157. Điều khoản chuyển tiếp\n1. Các hợp đ...,Chương VII ĐIỀU KHOẢN THI HÀNH,Chương VII ĐIỀU KHOẢN THI HÀNH\nĐiều 155. Sửa ...,7,Tóm tắt Điều 157 - Điều khoản chuyển tiếp:\n\n...,True,2025-03-31T12:40:24.222583,Luật Kinh Doanh Bảo Hiểm,Tóm tắt Điều 157 - Điều khoản chuyển tiếp:\n\n...


In [45]:
#EMBEDDING
embeddings = huggingface_embedding(luat_kd_bao_hiem_document_embedding_huggingface['embedding_input'], VIETNAMESE_DOCUMENT_EMBEDDING_MODEL)
#Check Embedding Vector Dimension
embeddings.shape

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

(157, 768)

In [46]:
#ADD EMBEDDING VECTOR BACK TO TABLE'
luat_kd_bao_hiem_document_embedding_huggingface['vietnamese_document_embedding'] = embeddings.tolist()

In [47]:
#FINAL_CHECKING
luat_kd_bao_hiem_document_embedding_huggingface.tail()

,reference_id,article_index,article_headline,article_content,section_title,section_content,section_id,generated_context,processing_success,processing_timestamp,document_name,embedding_input,vietnamese_document_embedding
152,L_08_2022_QH_15_D153,153,Điều 153. Quyền yêu cầu cung cấp thông tin của...,Điều 153. Quyền yêu cầu cung cấp thông tin của...,Chương VI QUẢN LÝ NHÀ NƯỚC VỀ HOẠT ĐỘNG KINH D...,Chương VI QUẢN LÝ NHÀ NƯỚC VỀ HOẠT ĐỘNG KINH D...,6,Tóm tắt:\n\nĐiều 153 quy định quyền hạn của Bộ...,True,2025-03-31T12:39:58.770351,Luật Kinh Doanh Bảo Hiểm,Tóm tắt:\n\nĐiều 153 quy định quyền hạn của Bộ...,"[-0.06018950045108795, 0.0026294677518308163, ..."
153,L_08_2022_QH_15_D154,154,Điều 154. Thanh tra hoạt động kinh doanh bảo hiểm,Điều 154. Thanh tra hoạt động kinh doanh bảo h...,Chương VI QUẢN LÝ NHÀ NƯỚC VỀ HOẠT ĐỘNG KINH D...,Chương VI QUẢN LÝ NHÀ NƯỚC VỀ HOẠT ĐỘNG KINH D...,6,Tóm tắt:\n\nĐiều 154 quy định về thanh tra chu...,True,2025-03-31T12:40:10.212626,Luật Kinh Doanh Bảo Hiểm,Tóm tắt:\n\nĐiều 154 quy định về thanh tra chu...,"[-0.029400072991847992, 0.0021093704272061586,..."
154,L_08_2022_QH_15_D155,155,"Điều 155. Sửa đổi, bổ sung Phụ lục IV của Luật...","Điều 155. Sửa đổi, bổ sung Phụ lục IV của Luật...",Chương VII ĐIỀU KHOẢN THI HÀNH,Chương VII ĐIỀU KHOẢN THI HÀNH\nĐiều 155. Sửa ...,7,Tóm tắt:\n\nĐiều luật bổ sung hai ngành nghề đ...,True,2025-03-31T12:40:14.731740,Luật Kinh Doanh Bảo Hiểm,Tóm tắt:\n\nĐiều luật bổ sung hai ngành nghề đ...,"[-0.08255720883607864, -0.019949709996581078, ..."
155,L_08_2022_QH_15_D156,156,Điều 156. Hiệu lực thi hành,Điều 156. Hiệu lực thi hành\n1. Luật này có hi...,Chương VII ĐIỀU KHOẢN THI HÀNH,Chương VII ĐIỀU KHOẢN THI HÀNH\nĐiều 155. Sửa ...,7,Tóm tắt:\n\nĐiều 156 quy định hiệu lực thi hàn...,True,2025-03-31T12:40:19.133501,Luật Kinh Doanh Bảo Hiểm,Tóm tắt:\n\nĐiều 156 quy định hiệu lực thi hàn...,"[-0.006480732001364231, 0.03075791895389557, -..."
156,L_08_2022_QH_15_D157,157,Điều 157. Điều khoản chuyển tiếp,Điều 157. Điều khoản chuyển tiếp\n1. Các hợp đ...,Chương VII ĐIỀU KHOẢN THI HÀNH,Chương VII ĐIỀU KHOẢN THI HÀNH\nĐiều 155. Sửa ...,7,Tóm tắt Điều 157 - Điều khoản chuyển tiếp:\n\n...,True,2025-03-31T12:40:24.222583,Luật Kinh Doanh Bảo Hiểm,Tóm tắt Điều 157 - Điều khoản chuyển tiếp:\n\n...,"[-0.030395401641726494, 0.051110200583934784, ..."


In [48]:
##SAVE TO PARQUET
saved_document = luat_kd_bao_hiem_document_embedding_huggingface.copy()
saved_document.to_parquet("/content/drive/MyDrive/COS30049 - PROJECT/Dataset/luat-kdbh.parquet")

##**PROCESS NGHỊ ĐỊNH 46/2023 VỀ ÁP DỤNG LUẬT KINH DOANH BẢO HIỂM (ND_46_2023_CP)**

In [ ]:
#LOAD PARQUET (IF ALREADY HAVE SAVED PROGRESS)
df_loaded = pd.read_parquet("/content/drive/MyDrive/COS30049 - PROJECT/Dataset/luat-kdbh.parquet")

In [54]:
#LOAD PDF
loaded_document = load_pdf("/content/drive/MyDrive/COS30049 - PROJECT/Dataset/thong tu/46_2023_ND-CP_m_571445.pdf")

In [55]:
#SPLIT DOCUMENT INTO ARTCLES AND GENERATE CONTEXT FOR EACH ARTICLE
document_name = "Nghị định 46"
document_doc_code = "ND_46_2023_CP"
document_articles = create_article_list_with_section(api_key=ANTHROPIC_API_KEY,document=loaded_document,document_name=document_name,document_doc_code=document_doc_code)

Starting processing: 123 articles across 8 sections
Already processed: 0 articles
Processing section 1/8: 'Chương I NHỮNG QUY ĐỊNH CHUNG' - 10 articles
Processing article 1/10: 'Điều 1. Phạm vi điều chỉnh...'
Processing article 2/10: 'Điều 2. Đối tượng áp dụng...'
Processing article 3/10: 'Điều 3. Nghiệp vụ bảo hiểm nhân thọ...'
Completed batch. Waiting 5s before next batch...
Processing article 4/10: 'Điều 4. Nghiệp vụ bảo hiểm phi nhân thọ...'
Processing article 5/10: 'Điều 5. Nghiệp vụ bảo hiểm sức khỏe...'
Processing article 6/10: 'Điều 6. Nguyên tắc xây dựng, quản lý, kh...'
Completed batch. Waiting 5s before next batch...
Processing article 7/10: 'Điều 7. Thông tin trong Cơ sở dữ liệu về...'
Processing article 8/10: 'Điều 8. Thu thập thông tin trong Cơ sở d...'
Processing article 9/10: 'Điều 9. Sử dụng thông tin từ Cơ sở dữ li...'
Completed batch. Waiting 5s before next batch...
Processing article 10/10: 'Điều 10. Kết nối Cơ sở dữ liệu về hoạt đ...'
Checkpoint saved: /content/dri

In [56]:
document_articles.tail()

,reference_id,article_index,article_headline,article_content,section_title,section_content,section_id,generated_context,processing_success,processing_timestamp,document_name
118,ND_46_2023_CP_D119,119,Điều 119. Rút trước tài khoản bảo hiểm hưu trí,Điều 119. Rút trước tài khoản bảo hiểm hưu trí...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,7,Tóm tắt:\n\nĐiều 119 quy định các trường hợp n...,True,2025-03-31T14:40:13.154875,Nghị định 46
119,ND_46_2023_CP_D120,120,Điều 120. Chuyển tài khoản bảo hiểm hưu trí,Điều 120. Chuyển tài khoản bảo hiểm hưu trí\n1...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,7,Tóm tắt:\n\nĐiều 120 quy định quyền chuyển như...,True,2025-03-31T14:40:48.088594,Nghị định 46
120,ND_46_2023_CP_D121,121,Điều 121. Tạm thời đóng tài khoản bảo hiểm hưu...,Điều 121. Tạm thời đóng tài khoản bảo hiểm hưu...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,7,Tóm tắt:\n\nĐiều khoản quy định về việc tạm th...,True,2025-03-31T14:41:20.479235,Nghị định 46
121,ND_46_2023_CP_D122,122,Điều 122. Hiệu lực thi hành,Điều 122. Hiệu lực thi hành\n1. Nghị định này ...,Chương VIII ĐIỀU KHOẢN THI HÀNH,Chương VIII ĐIỀU KHOẢN THI HÀNH\nĐiều 122. Hiệ...,8,Tóm tắt Điều 122 - Hiệu lực thi hành:\n\nNghị ...,True,2025-03-31T14:41:34.773677,Nghị định 46
122,ND_46_2023_CP_D123,123,Điều 123. Trách nhiệm thi hành,"Điều 123. Trách nhiệm thi hành\nCác Bộ trưởng,...",Chương VIII ĐIỀU KHOẢN THI HÀNH,Chương VIII ĐIỀU KHOẢN THI HÀNH\nĐiều 122. Hiệ...,8,Tóm tắt:\n\nĐiều 123 quy định trách nhiệm pháp...,True,2025-03-31T14:41:39.152487,Nghị định 46


In [57]:
#SETUP MODEL
VIETNAMESE_DOCUMENT_EMBEDDING_MODEL = 'dangvantuan/vietnamese-document-embedding'
#PREPARE DATA
nd46_document_embedding_huggingface = document_articles.copy()
nd46_document_embedding_huggingface["embedding_input"] = nd46_document_embedding_huggingface.apply(
        lambda row: f"{row['generated_context']}\n\n{row['article_content']}", axis=1
    )
nd46_document_embedding_huggingface["embedding_input"].tolist()
#CHECKING DATA BEFORE EMBEDDING
nd46_document_embedding_huggingface.tail()

,reference_id,article_index,article_headline,article_content,section_title,section_content,section_id,generated_context,processing_success,processing_timestamp,document_name,embedding_input
118,ND_46_2023_CP_D119,119,Điều 119. Rút trước tài khoản bảo hiểm hưu trí,Điều 119. Rút trước tài khoản bảo hiểm hưu trí...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,7,Tóm tắt:\n\nĐiều 119 quy định các trường hợp n...,True,2025-03-31T14:40:13.154875,Nghị định 46,Tóm tắt:\n\nĐiều 119 quy định các trường hợp n...
119,ND_46_2023_CP_D120,120,Điều 120. Chuyển tài khoản bảo hiểm hưu trí,Điều 120. Chuyển tài khoản bảo hiểm hưu trí\n1...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,7,Tóm tắt:\n\nĐiều 120 quy định quyền chuyển như...,True,2025-03-31T14:40:48.088594,Nghị định 46,Tóm tắt:\n\nĐiều 120 quy định quyền chuyển như...
120,ND_46_2023_CP_D121,121,Điều 121. Tạm thời đóng tài khoản bảo hiểm hưu...,Điều 121. Tạm thời đóng tài khoản bảo hiểm hưu...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,7,Tóm tắt:\n\nĐiều khoản quy định về việc tạm th...,True,2025-03-31T14:41:20.479235,Nghị định 46,Tóm tắt:\n\nĐiều khoản quy định về việc tạm th...
121,ND_46_2023_CP_D122,122,Điều 122. Hiệu lực thi hành,Điều 122. Hiệu lực thi hành\n1. Nghị định này ...,Chương VIII ĐIỀU KHOẢN THI HÀNH,Chương VIII ĐIỀU KHOẢN THI HÀNH\nĐiều 122. Hiệ...,8,Tóm tắt Điều 122 - Hiệu lực thi hành:\n\nNghị ...,True,2025-03-31T14:41:34.773677,Nghị định 46,Tóm tắt Điều 122 - Hiệu lực thi hành:\n\nNghị ...
122,ND_46_2023_CP_D123,123,Điều 123. Trách nhiệm thi hành,"Điều 123. Trách nhiệm thi hành\nCác Bộ trưởng,...",Chương VIII ĐIỀU KHOẢN THI HÀNH,Chương VIII ĐIỀU KHOẢN THI HÀNH\nĐiều 122. Hiệ...,8,Tóm tắt:\n\nĐiều 123 quy định trách nhiệm pháp...,True,2025-03-31T14:41:39.152487,Nghị định 46,Tóm tắt:\n\nĐiều 123 quy định trách nhiệm pháp...


In [58]:
#EMBEDDING
embeddings = huggingface_embedding(nd46_document_embedding_huggingface['embedding_input'], VIETNAMESE_DOCUMENT_EMBEDDING_MODEL)
#Check Embedding Vector Dimension
embeddings.shape

Batches:   0%|          | 0/31 [00:00<?, ?it/s]

(123, 768)

In [59]:
#ADD EMBEDDING VECTOR BACK TO TABLE'
nd46_document_embedding_huggingface['vietnamese_document_embedding'] = embeddings.tolist()
#FINAL_CHECKING
nd46_document_embedding_huggingface.tail()

,reference_id,article_index,article_headline,article_content,section_title,section_content,section_id,generated_context,processing_success,processing_timestamp,document_name,embedding_input,vietnamese_document_embedding
118,ND_46_2023_CP_D119,119,Điều 119. Rút trước tài khoản bảo hiểm hưu trí,Điều 119. Rút trước tài khoản bảo hiểm hưu trí...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,7,Tóm tắt:\n\nĐiều 119 quy định các trường hợp n...,True,2025-03-31T14:40:13.154875,Nghị định 46,Tóm tắt:\n\nĐiều 119 quy định các trường hợp n...,"[-0.055266089737415314, 0.028841039165854454, ..."
119,ND_46_2023_CP_D120,120,Điều 120. Chuyển tài khoản bảo hiểm hưu trí,Điều 120. Chuyển tài khoản bảo hiểm hưu trí\n1...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,7,Tóm tắt:\n\nĐiều 120 quy định quyền chuyển như...,True,2025-03-31T14:40:48.088594,Nghị định 46,Tóm tắt:\n\nĐiều 120 quy định quyền chuyển như...,"[-0.03540479391813278, 0.04364600032567978, -0..."
120,ND_46_2023_CP_D121,121,Điều 121. Tạm thời đóng tài khoản bảo hiểm hưu...,Điều 121. Tạm thời đóng tài khoản bảo hiểm hưu...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,Chương VII NGHIỆP VỤ BẢO HIỂM LIÊN KẾT ĐẦU TƯ ...,7,Tóm tắt:\n\nĐiều khoản quy định về việc tạm th...,True,2025-03-31T14:41:20.479235,Nghị định 46,Tóm tắt:\n\nĐiều khoản quy định về việc tạm th...,"[-0.07418669760227203, -0.002445597667247057, ..."
121,ND_46_2023_CP_D122,122,Điều 122. Hiệu lực thi hành,Điều 122. Hiệu lực thi hành\n1. Nghị định này ...,Chương VIII ĐIỀU KHOẢN THI HÀNH,Chương VIII ĐIỀU KHOẢN THI HÀNH\nĐiều 122. Hiệ...,8,Tóm tắt Điều 122 - Hiệu lực thi hành:\n\nNghị ...,True,2025-03-31T14:41:34.773677,Nghị định 46,Tóm tắt Điều 122 - Hiệu lực thi hành:\n\nNghị ...,"[-0.012344062328338623, 0.03636099770665169, -..."
122,ND_46_2023_CP_D123,123,Điều 123. Trách nhiệm thi hành,"Điều 123. Trách nhiệm thi hành\nCác Bộ trưởng,...",Chương VIII ĐIỀU KHOẢN THI HÀNH,Chương VIII ĐIỀU KHOẢN THI HÀNH\nĐiều 122. Hiệ...,8,Tóm tắt:\n\nĐiều 123 quy định trách nhiệm pháp...,True,2025-03-31T14:41:39.152487,Nghị định 46,Tóm tắt:\n\nĐiều 123 quy định trách nhiệm pháp...,"[-0.09930054098367691, 0.0062782661989331245, ..."


In [60]:
##SAVE TO PARQUET
saved_document = nd46_document_embedding_huggingface.copy()
saved_document.to_parquet("/content/drive/MyDrive/COS30049 - PROJECT/Dataset/ND_46_2023_CP.parquet")

##**PROCESS THUYET MINH TAI CHINH BAO VIET**

In [8]:
load_df = pd.read_csv('/content/drive/MyDrive/COS30049 - PROJECT/Dataset/financial_report.csv')

In [31]:
import pandas as pd
def generate_reference_id(row, doc_code):
  """Generates a reference ID based on the row data and document code."""
  chunk_id_num = int(row['chunk_id'].split('_')[-1])  # Extract the chunk ID number
  return f"{doc_code}_Q{row['quarter']}_{row['year']}_C{chunk_id_num}"

def modify_dataframe(df, document_name,doc_code):
  """Renames columns and adds section_id column to the dataframe."""
  # Rename columns
  df = df.rename(columns={
      'content': 'article_content',
      'source': 'document_name',
      'section': 'section_title',
      'note': 'article_headline'
  })
  """Adds a reference ID column to the dataframe."""
  df['reference_id'] = df.apply(lambda row: generate_reference_id(row, doc_code), axis=1)
  # Add section_id column
  df['section_id'] = df.apply(lambda row: f"Q{row['quarter']}{row['year']}", axis=1)
  # Set document_name column value
  df['document_name'] = document_name
  # Drop chunk_id column
  df = df.drop(columns=['chunk_id'])

  # Move reference_id to the first column
  cols = df.columns.tolist()
  cols.insert(0, cols.pop(cols.index('reference_id')))
  df = df.reindex(columns=cols)
  return df

# Example usage
doc_code = "BVH_TM_BCTC"
document_name = "Thuyết minh BCTC Bảo Việt"
load_df = modify_dataframe(load_df, document_name, doc_code)

In [33]:
#SETUP MODEL
VIETNAMESE_DOCUMENT_EMBEDDING_MODEL = 'dangvantuan/vietnamese-document-embedding'
#PREPARE DATA
bv_tm_bctc_document_embedding_huggingface = load_df.copy()
#CHECKING DATA BEFORE EMBEDDING
bv_tm_bctc_document_embedding_huggingface.tail()

,reference_id,article_content,document_name,section_title,article_headline,section_type,chunk_type,quarter,year,section_id
111,BVH_TM_BCTC_Q4_2024_C54,35. THÔNG TIN THEO BỘ PHẬN\n\n35. THÔNG TIN TH...,Thuyết minh BCTC Bảo Việt,THUYẾT MINH BÁO CÁO TÀI CHÍNH HỢP NHẤT,35. THÔNG TIN THEO BỘ PHẬN,note_item,financial_note,4,2024,Q42024
112,BVH_TM_BCTC_Q4_2024_C55,38. THÔNG TIN THEO BỘ PHẬN\n\n38. THÔNG TIN TH...,Thuyết minh BCTC Bảo Việt,THUYẾT MINH BÁO CÁO TÀI CHÍNH HỢP NHẤT,38. THÔNG TIN THEO BỘ PHẬN,note_item,financial_note,4,2024,Q42024
113,BVH_TM_BCTC_Q4_2024_C56,36. CÁC CAM KÉT VÀ CÁC TÀI SÀN TIỀM TÀNG\n\n36...,Thuyết minh BCTC Bảo Việt,THUYẾT MINH BÁO CÁO TÀI CHÍNH HỢP NHẤT,36. CÁC CAM KÉT VÀ CÁC TÀI SÀN TIỀM TÀNG,note_item,financial_note,4,2024,Q42024
114,BVH_TM_BCTC_Q4_2024_C57,36. CÁC CAM KÉT VÀ CÁC TÀI SẢN TIỀM TÀNG\n\n36...,Thuyết minh BCTC Bảo Việt,THUYẾT MINH BÁO CÁO TÀI CHÍNH HỢP NHẤT,36. CÁC CAM KÉT VÀ CÁC TÀI SẢN TIỀM TÀNG,note_item,financial_note,4,2024,Q42024
115,BVH_TM_BCTC_Q4_2024_C58,38. CÁC SỰ KIỆN PHÁT SINH SAU NGÀY KÉT THÚC KỲ...,Thuyết minh BCTC Bảo Việt,THUYẾT MINH BÁO CÁO TÀI CHÍNH HỢP NHẤT,38. CÁC SỰ KIỆN PHÁT SINH SAU NGÀY KÉT THÚC KỲ...,note_item,financial_note,4,2024,Q42024


In [34]:
#EMBEDDING
embeddings = huggingface_embedding(bv_tm_bctc_document_embedding_huggingface['article_content'], VIETNAMESE_DOCUMENT_EMBEDDING_MODEL)
#ADD EMBEDDING VECTOR BACK TO TABLE'
bv_tm_bctc_document_embedding_huggingface['vietnamese_document_embedding'] = embeddings.tolist()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.09k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

NameError: name 'nd46_document_embedding_hubv_tm_bctc_document_embedding_huggingfaceggingface' is not defined

In [38]:
#Check Embedding Vector Dimension
embeddings.shape

(116, 768)

In [37]:
#SAVE TO PARQUET
saved_doc = bv_tm_bctc_document_embedding_huggingface.copy()
saved_doc.to_parquet("/content/drive/MyDrive/COS30049 - PROJECT/Dataset/bv_tm_bctc_2024.parquet")

##**EXAMPLE WORKFLOW**

In [2]:
#Load document from file path
document = load_pdf(file_path)

#Split document into Section
document_section = split_document_into_sections(document)

#Generate Document Article with generate context
document_article = create_article_list_with_section(api_key=ANTHROPIC_API_KEY,document=document_section)

#EMBEDDING

#DENSE VECTOR EMBEDDING
#SETUP MODEL
VIETNAMESE_DOCUMENT_EMBEDDING_MODEL = 'dangvantuan/vietnamese-document-embedding'
#PREPARE DATA
document_article["embedding_input"] = document_article.apply(
        lambda row: f"{row['generated_context']}\n\n{row['article_content']}", axis=1
    )
document_article["embedding_input"].tolist()
#Generate Dense Vector Embedding
dense_embedding = huggingface_embedding(document_article["embedding_input"],VIETNAMESE_DOCUMENT_EMBEDDING_MODEL)
document_article['vietnamese_document_embedding'] = dense_embedding.tolist()

#SPARSE VECTOR EMBEDDING
vectorizer, sparse_vecs = generate_sparse_vectors(document_article["embedding_input"])
document_article["sparse_vector"] = sparse_vecs

#SAVE DATAFRAME TO PARQUET FUR FURTHER USE/ CHECKPOINT
document_article.to_parquet(file_path)
#SAVE VECTORIZER FOR USING IN RETRIEVAL PART
joblib.dump(vectorizer, vectorizer_path)

#**INSERT ARTICLE CHUNKS TO DATABASE**

##**INSTALL QDRANT DATABASE**

In [10]:
!pip install --upgrade qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.7/306.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.8 MB/s eta 0:00:00


##**INITIALIZE DATABASE**

In [8]:
QDRANT_URL = "https://fcca288e-0b3a-4303-be81-ea6b08281340.us-east-1-0.aws.cloud.qdrant.io:6333"

In [25]:
#Init client
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance,SparseVectorParams

client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY
)

##**CREATE DATABASE TABLE (ONLY IF HAVENT CREATED)**

In [7]:
client.create_collection(
    collection_name="insurance_policy_67_vietnamese_embedding",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE)
)

<ipython-input-7-a46c12eaf351>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

##**LOAD DATA**

In [43]:
import pandas as pd
df_loaded = pd.read_parquet("/content/drive/MyDrive/COS30049 - PROJECT/Dataset/bv_tm_bctc_2024.parquet")
df = df_loaded.copy()

###**VECTOR EMBEDDING VALID CHECKING**

In [44]:
assert all(len(vec) == 768 for vec in df["vietnamese_document_embedding"]), "🚨 Some embeddings are not 768-dim!"
# assert all(len(vec) == 1024 for vec in df["vietnamese_document_embedding"]), "🚨 Some embeddings are not 1024-dim!"
bad_rows = df[df["vietnamese_document_embedding"].apply(lambda x: len(x) != 768)]
print(f"⚠️ Found {len(bad_rows)} bad embeddings:")
display(bad_rows[["reference_id", "vietnamese_document_embedding"]].head())
df["embedding_length"] = df["vietnamese_document_embedding"].apply(lambda x: len(x))
df["embedding_length"].value_counts()

⚠️ Found 0 bad embeddings:


,reference_id,vietnamese_document_embedding


,count
embedding_length,
768,116


##**UPLOAD DATA TO DATABASE**

**UPLOAD DATAFRAME TO DATABASE**

In [17]:
import uuid
from qdrant_client.models import PointStruct
points = [
    PointStruct(
        id=str(uuid.uuid4()),
        vector=row["vietnamese_document_embedding"],
        payload={
            # Identifiers & Structure
            "reference_id": row["reference_id"],
            "document_name": row["document_name"],
            "section_id": row["section_id"],
            "section_title": row["section_title"],
            "article_headline": row["article_headline"],

            # Article Content
            "content": row["article_content"],
            "article_context": row["generated_context"],
        }
    )
    for _, row in df.iterrows()
]

client.upload_points(collection_name="insurance_policy_67_vietnamese_embedding", points=points)

**INSERT (UPDATE) DATAFRAME TO DATABASE**

In [46]:
#INSERT TO DATABASE (UPDATE)
import uuid
from qdrant_client.models import PointStruct
points = [
    PointStruct(
        id=str(uuid.uuid4()),
        vector=row["vietnamese_document_embedding"],
        payload={
            # Identifiers & Structure
            "reference_id": row["reference_id"],
            "document_name": row["document_name"],
            "section_id": row["section_id"],
            "section_title": row["section_title"],
            "article_headline": row["article_headline"],

            # Article Content
            "content": row["article_content"],
            "article_context": row["generated_context"],
        }
    )
    for _, row in df.iterrows()
]

client.upsert(collection_name="insurance_policy_67_vietnamese_embedding", points=points)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

In [29]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, SparseVector, VectorParams, Distance
import pandas as pd
import joblib

# Step 1: Retrieve all data from Qdrant

all_points = []
scroll_offset = None

while True:
    points, scroll_offset = client.scroll(
        collection_name="insurance_policy_67_vietnamese_embedding",
        limit=100,
        offset=scroll_offset,
        with_vectors=True,
        with_payload=True
    )
    all_points.extend(points)
    if scroll_offset is None:
        break

# Step 2: Convert to DataFrame
records = []
for pt in all_points:
    record = {
        "id": pt.id,
        "reference_id": pt.payload.get("reference_id", ""),
        "document_name": pt.payload.get("document_name", ""),
        "section_id": pt.payload.get("section_id", ""),
        "section_title": pt.payload.get("section_title", ""),
        "article_headline": pt.payload.get("article_headline", ""),
        "content": pt.payload.get("content", ""),
        "article_context": pt.payload.get("article_context", ""),
        "dense_vector": pt.vector.get("dense") if isinstance(pt.vector, dict) else pt.vector
    }
    records.append(record)

df = pd.DataFrame(records)

# Step 3: Create 'embedding_input' = content + article_context (with fallback to "")
df["article_context"] = df["article_context"].fillna("")
df["embedding_input"] = df["content"] + " " + df["article_context"]

# Step 4: Generate sparse vectors using TfidfVectorizer
embedding_inputs = df["embedding_input"].tolist()
vectorizer, sparse_vecs = generate_sparse_vectors(embedding_inputs)
df["sparse_vector"] = sparse_vecs



In [30]:
display(df.head())

,id,reference_id,document_name,section_id,section_title,article_headline,content,article_context,dense_vector,embedding_input,sparse_vector
0,013c569e-be1f-426d-886e-4155c5edfe1a,TT_50_2022_BTC_D26,Thông tư 50,2,Chương II QUY ĐỊNH CỤ THỂ,Điều 26. Nguyên tắc bồi thường bảo hiểm,Điều 26. Nguyên tắc bồi thường bảo hiểm\n1. Kh...,Tóm tắt:\n\nNguyên tắc bồi thường bảo hiểm tai...,"[-0.023466775, -0.01755457, -0.034860507, 0.03...",Điều 26. Nguyên tắc bồi thường bảo hiểm\n1. Kh...,"{'indices': [3737, 444, 2695, 3495, 1713, 3249..."
1,0217fcb4-cb2d-4304-ba54-9cffeab02530,BVH_TM_BCTC_Q3_2024_C19,Thuyết minh BCTC Bảo Việt,Q32024,THUYẾT MINH BÁO CÁO TÀI CHÍNH HỢP NHẤT,4. TÓM TÁT CÁC CHÍNH SÁCH KÉ TOÁN CHỦ YẾU,4. TÓM TÁT CÁC CHÍNH SÁCH KÉ TOÁN CHỦ YẾU\n\n4...,4. TÓM TÁT CÁC CHÍNH SÁCH KÉ TOÁN CHỦ YẾU\n\n4...,"[-0.07390901, -0.015425013, 0.014627502, 0.027...",4. TÓM TÁT CÁC CHÍNH SÁCH KÉ TOÁN CHỦ YẾU\n\n4...,"{'indices': [444, 1687, 2170, 2311, 2985, 3859..."
2,022f5fe9-f36a-4d4e-a30a-99dcb2d10a98,TT_67_2023_BTC_D8,Thông tư 67,2,"Chương II CUNG CẤP DỊCH VỤ, SẢN PHẨM BẢO HIỂM ...","Điều 8. Trách nhiệm của doanh nghiệp bảo hiểm,...","Điều 8. Trách nhiệm của doanh nghiệp bảo hiểm,...",Tóm tắt:\n\nQuy định trách nhiệm của doanh ngh...,"[-0.087716974, -0.050677326, -0.013212912, -0....","Điều 8. Trách nhiệm của doanh nghiệp bảo hiểm,...","{'indices': [3737, 1713, 3249, 1687, 2170, 231..."
3,02f1ccf8-26e4-4897-9c7d-433a52b136cb,L_08_2022_QH_15_D137,Luật Kinh Doanh Bảo Hiểm,4,"Chương IV ĐẠI LÝ BẢO HIỂM, DOANH NGHIỆP MÔI GI...",Điều 137. Quyền và nghĩa vụ của doanh nghiệp m...,Điều 137. Quyền và nghĩa vụ của doanh nghiệp m...,Tóm tắt:\n\nĐiều 137 quy định quyền và nghĩa v...,"[-0.050222665, -0.007957419, -0.019009147, 0.0...",Điều 137. Quyền và nghĩa vụ của doanh nghiệp m...,"{'indices': [3737, 2695, 3495, 1713, 3249, 168..."
4,03562f9f-c313-4531-9830-d9b03a499c42,BVH_TM_BCTC_Q3_2024_C28,Thuyết minh BCTC Bảo Việt,Q32024,THUYẾT MINH BÁO CÁO TÀI CHÍNH HỢP NHẤT,14. CÁC KHOẢN ĐẦU TỰ TÀI CHÍNH,14. CÁC KHOẢN ĐẦU TỰ TÀI CHÍNH\n\n14. CÁC KHOẢ...,14. CÁC KHOẢN ĐẦU TỰ TÀI CHÍNH\n\n14. CÁC KHOẢ...,"[-0.025095804, 0.0024317813, 0.016748559, 0.00...",14. CÁC KHOẢN ĐẦU TỰ TÀI CHÍNH\n\n14. CÁC KHOẢ...,"{'indices': [3737, 444, 1687, 2170, 2311, 1890..."


In [31]:
# Step 5: Save df to parquet and vectorizer to pickle
df_path = "/content/drive/MyDrive/COS30049 - PROJECT/Dataset/full_rag_chatbot_dataset.parquet"
vectorizer_path = "/content/drive/MyDrive/COS30049 - PROJECT/Dataset/tfidf_vectorizer.pkl"
# Explicitly set the 'section_id' column dtype to string before saving
df['section_id'] = df['section_id'].astype(str)  # Ensure 'section_id' is of type string
df.to_parquet(df_path)
joblib.dump(vectorizer, vectorizer_path)

['/content/drive/MyDrive/COS30049 - PROJECT/Dataset/tfidf_vectorizer.pkl']

In [37]:
# Step 6: Recreate the collection to include both dense and sparse vectors
client.recreate_collection(
    collection_name="full_rag_legal_docs",
    vectors_config={
        "dense": VectorParams(size=768, distance=Distance.COSINE)
    },
    sparse_vectors_config={"sparse": SparseVectorParams()},
)

# Step 7: Upload new database to Qdrant
# ✅ Build points with both dense & sparse vectors
points = [
    PointStruct(
        id=row["id"],
        vector={
            "dense": row["dense_vector"],  # dense vector
            "sparse": SparseVector(**row["sparse_vector"])  # sparse vector (TF-IDF)
        },
        payload={
            # Identifiers & Structure
            "reference_id": row["reference_id"],
            "document_name": row["document_name"],
            "section_id": row["section_id"],
            "section_title": row["section_title"],
            "article_headline": row["article_headline"],

            # Article Content
            "content": row["content"],
            "article_context": row["article_context"],
        }
    )
    for _, row in df.iterrows()
]

client.upload_points(collection_name="full_rag_legal_docs", points=points)

print("✅ Successfully uploaded dense + sparse hybrid vectors to Qdrant.")


<ipython-input-37-dafd60413da9>:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


✅ Successfully uploaded dense + sparse hybrid vectors to Qdrant.
